### Get openai embeddings for categories

##### Will need openai API

In [2]:
import os, time, pandas as pd
from openai import OpenAI
import tiktoken

In [3]:
# ----------------- CONFIG -----------------
client = OpenAI(api_key="") # insert api key
EMBED_MODEL = "text-embedding-3-small"

In [4]:
df = pd.read_csv('combined_train.csv')
df

,report_id,company_name,isin,reported_at,pdf_path,pages_path,blocks_path,year_month,DATE,SYMBOL,...,TICKER,SECTOR,MOMENTUM,VALUE,SIZE,BETA,SEDOL,ISIN,RETURNS,report_type
0,f_4csLsjAhudR2mcDosUbKD5,LULULEMON ATHLETICA INC,US5500211090,2024-03-21,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,2024-03,2024-03-28,55002110,...,LULU,Consumer Discretionary,44.9034,-0.609,10.840906,1.217020,B23FN39,US5500211090,-0.163652,Annual
1,f_9tgeLS94nO4peRbBEIbvje,LULULEMON ATHLETICA INC,US5500211090,2024-08-29,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,2024-08,2024-08-30,55002110,...,LULU,Consumer Discretionary,-32.0468,-0.131,10.408057,0.576961,B23FN39,US5500211090,0.003132,Quarterly
2,f_PDplDokn,LULULEMON ATHLETICA INC,US5500211090,2023-08-31,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,2023-08,2023-08-31,55002110,...,LULU,Consumer Discretionary,26.1935,-0.758,10.815705,1.478690,B23FN39,US5500211090,0.007212,Quarterly
3,f_WoQLjVEq,LULULEMON ATHLETICA INC,US5500211090,2023-12-07,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,2023-12,2023-12-29,55002110,...,LULU,Consumer Discretionary,38.3593,-0.938,11.110417,1.365380,B23FN39,US5500211090,0.144338,Quarterly
4,f_cC5GK9NDnFMWH7KTPOIlox,LULULEMON ATHLETICA INC,US5500211090,2024-12-05,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,2024-12,2024-12-31,55002110,...,LULU,Consumer Discretionary,-37.2841,-0.506,10.794169,-0.378504,B23FN39,US5500211090,0.192572,Quarterly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4241,o_f9saaXvCQe9wIyNvILHS3l,MOODY'S CORP,US6153691059,2023-07-06,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,2023-07,2023-07-31,61536910,...,MCO,Financials,13.1608,-0.850,11.078349,1.627940,2252058,US6153691059,0.014466,Event/Other
4242,o_fwe9aJYEVTFtxduTAdK6sd,GE VERNOVA INC,US36828A1016,2024-12-10,data/reports/GE_VERNOVA_INC/o_fwe9aJYEVTFtxduT...,data/reports/GE_VERNOVA_INC/o_fwe9aJYEVTFtxduT...,data/reports/GE_VERNOVA_INC/o_fwe9aJYEVTFtxduT...,2024-12,2024-12-31,36828A10,...,GEV,Industrials,NaN,-0.803,11.419023,0.218221,BP6H4Y1,US36828A1016,-0.014815,Event/Other
4243,o_gAqntEyHWJXJDmnXkTAQEe,BANK OF AMERICA CORP,US0605051046,2024-09-30,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,2024-09,2024-09-30,06050510,...,BAC,Financials,51.9570,2.520,12.630449,1.355190,2295677,US0605051046,-0.019726,Earnings
4244,o_gdd49aPyFdkFEH34pnybsJ,CITIGROUP INC,US1729674242,2024-06-18,data/reports/CITIGROUP_INC/o_gdd49aPyFdkFEH34p...,data/reports/CITIGROUP_INC/o_gdd49aPyFdkFEH34p...,data/reports/CITIGROUP_INC/o_gdd49aPyFdkFEH34p...,2024-06,2024-06-28,17296742,...,C,Financials,40.8893,3.993,11.673367,1.276030,2297907,US1729674242,0.018456,Event/Other


In [5]:
# ============== DICTIONARY ==============
finance_dict = {
    "Sales/Revenue": [
        "sales", "revenue", "turnover", "top line", "net sales", "gross sales",
        "service income", "product revenue", "subscription revenue", "customer receipts",
        "contract revenue", "retail revenue", "distribution income", "sales growth",
        "sales volume", "sales pipeline"
    ],
    "Expense/Cost": [
        "operating expenses", "cost of goods sold", "COGS", "administrative expenses",
        "selling expenses", "marketing spend", "R&D expense", "depreciation expense",
        "amortization", "general and administrative costs", "overheads", "freight cost",
        "utilities expense", "rent expense", "interest expense", "insurance expense"
    ],
    "Profit/Loss": [
        "profit", "net income", "bottom line", "earnings", "EBIT", "EBITDA",
        "operating profit", "gross margin", "net margin", "loss", "deficit",
        "comprehensive income", "profitability", "earnings per share", "EPS"
    ],
    "Operations": [
        "operational efficiency", "supply chain operations", "production process",
        "manufacturing operations", "operational costs", "capacity utilization",
        "day-to-day operations", "operating metrics", "factory operations",
        "inventory management", "logistics operations", "fulfillment", "distribution center"
    ],
    "Liquidity": [
        "liquidity", "cash balance", "working capital", "short-term assets",
        "short-term liabilities", "current ratio", "quick ratio", "liquid assets",
        "cash flow availability", "operating liquidity", "cash equivalents",
        "treasury position", "liquidity buffer"
    ],
    "Investment": [
        "capital expenditures", "CAPEX", "strategic investment", "R&D investment",
        "long-term investment", "property, plant, equipment", "acquisition",
        "venture investment", "equity investment", "infrastructure spending",
        "joint venture", "minority interest", "seed funding", "capital allocation"
    ],
    "Financing": [
        "debt issuance", "equity issuance", "financing activities", "loan facility",
        "bond offering", "capital raise", "share buyback", "dividends", "bridge loan",
        "credit facility", "refinancing", "leverage", "interest payments",
        "capital structure", "borrowings"
    ],
    "Litigation": [
        "lawsuit", "legal proceedings", "litigation", "court case", "settlement",
        "arbitration", "class action", "intellectual property dispute",
        "regulatory investigation", "legal liability", "compliance violation",
        "legal costs", "patent dispute", "pending litigation"
    ],
    "Employment": [
        "employee headcount", "hiring", "restructuring", "workforce reduction",
        "layoffs", "salary", "compensation", "wages", "benefits", "union negotiation",
        "employee turnover", "recruitment", "training programs", "stock options",
        "employee retention"
    ],
    "Regulation/Tax": [
        "tax expense", "income tax", "corporate tax", "tax compliance",
        "regulatory compliance", "government regulation", "tax liability",
        "environmental regulation", "tax provision", "statutory filings",
        "withholding tax", "customs duties", "tax audits", "tax reforms"
    ],
    "Accounting": [
        "accounting policies", "financial reporting", "GAAP", "IFRS", "restatement",
        "accruals", "audit", "balance sheet", "income statement", "consolidation",
        "impairment", "valuation", "internal controls", "bookkeeping",
        "financial disclosure"
    ]
}

In [6]:
# ============== BUSINESS DICTIONARY ==============
biz_dict = {
    "Prod": [
        "product", "goods", "merchandise", "inventory", "finished goods",
        "product line", "portfolio", "product features", "new product launch",
        "product design", "product innovation", "product lifecycle",
        "product differentiation"
    ],
    "Buyer": [
        "customer", "client", "consumer", "end user", "buyer",
        "customer base", "customer demand", "customer satisfaction",
        "customer acquisition", "customer loyalty", "user engagement",
        "repeat purchase", "churn rate"
    ],
    "Process": [
        "process", "procedure", "workflow", "business process",
        "standard operating procedure", "SOP", "best practices",
        "compliance procedure", "automation process", "quality control",
        "process optimization", "review process"
    ],
    "Loc": [
        "location", "geographic region", "market region", "facility",
        "plant", "office", "branch", "warehouse", "regional market",
        "domestic market", "international market", "headquarters",
        "subsidiary location"
    ],
    "Promo": [
        "promotion", "advertising", "marketing campaign", "brand awareness",
        "sales promotion", "discount", "special offer", "public relations",
        "digital marketing", "social media marketing", "event sponsorship",
        "advertising spend", "promotional activities"
    ],
    "Supplier": [
        "supplier", "vendor", "logistics", "distribution",
        "supply chain", "outsourcing", "third-party supplier",
        "procurement", "materials sourcing", "vendor relationship",
        "supplier contract", "supply disruption", "inventory sourcing"
    ],
    "Mgmt": [
        "management", "executives", "leadership", "directors",
        "board of directors", "shareholders", "ownership",
        "corporate governance", "chief executive officer",
        "chief financial officer", "CEO", "CFO", "stakeholders",
        "executive team"
    ],
    "Strategy": [
        "strategy", "business plan", "corporate strategy",
        "growth strategy", "long-term plan", "competitive strategy",
        "market entry", "business model", "strategic initiative",
        "operational strategy", "expansion strategy", "cost leadership",
        "differentiation strategy"
    ],
    "IndStruct": [
        "industry", "market structure", "competitive landscape",
        "competition", "market dynamics", "barriers to entry",
        "market share", "fragmented industry", "industry trend",
        "emerging industry", "sector growth", "industry benchmark"
    ],
    "BizDesc": [
        "business description", "company overview", "corporate profile",
        "business model", "mission statement", "vision statement",
        "organizational structure", "company history", "line of business",
        "nature of operations", "company background", "entity overview"
    ]
}

In [7]:
risk_dict = {
    "RiskFirm": [
        "company specific risk", "firm-specific risk", "business risk",
        "operational risk", "management risk", "financial reporting risk",
        "supply chain disruption", "loss of key customer", "loss of key supplier",
        "legal disputes", "product liability", "technology failure",
        "fraud risk", "internal control weakness", "governance risk"
    ],
    "RiskGeneral": [
        "market risk", "economic downturn", "industry risk",
        "political risk", "currency fluctuation", "inflation risk",
        "regulatory risk", "environmental risk", "climate change risk",
        "competition", "macroeconomic risk", "interest rate risk",
        "geopolitical instability", "pandemic risk", "natural disaster"
    ]
}

In [8]:
analyst_dict = {
    "Analyst": [
        "analyst estimates", "analyst forecast", "market analyst",
        "equity research", "research report", "consensus estimate",
        "analyst expectation", "third-party research", "sell-side analyst",
        "buy-side analyst", "independent analysis", "market commentary"
    ],
    "Company": [
        "management statement", "company statement", "press release",
        "CEO remarks", "executive commentary", "management discussion",
        "corporate outlook", "guidance provided", "board of directors statement",
        "chairman’s letter", "quarterly update", "earnings call remarks"
    ]
}

In [9]:
OUT_PATH_ALL = "category_embeddings.parquet"

def embed_dict(cat_dict, domain, all_rows):
    for cat, phrases in cat_dict.items():
        response = client.embeddings.create(
            model=EMBED_MODEL,
            input=phrases
        )
        for phrase, emb in zip(phrases, [item.embedding for item in response.data]):
            all_rows.append({
                "domain": domain,   # e.g., Finance / Business / Risk / Analyst
                "category": cat,    # e.g., Sales/Revenue, RiskFirm
                "phrase": phrase,
                "embedding": emb
            })

all_rows = []
embed_dict(finance_dict, "Finance", all_rows)
embed_dict(biz_dict, "Business", all_rows)
embed_dict(risk_dict, "Risk", all_rows)
embed_dict(analyst_dict, "Analyst", all_rows)

pd.DataFrame(all_rows).to_parquet(OUT_PATH_ALL, index=False)